<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/10_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

若我兩個df如下，想要將兩個df併再一起並且除col7外(col1-col6相同)的資料若相同，則視為一筆資料，且其col7就合併位元

In [2]:
import pandas as pd

# 範例資料
data1 = {
    'col1': ['A', 'B', 'C'],
    'col2': [1, 2, 3],
    'col3': ['X', 'Y', 'Z'],
    'col4': [10, 20, 30],
    'col5': [100, 200, 300],
    'col6': [1000, 2000, 3000],
    'col7': ['10000 00000 00000', '10000 00000 00000', '10000 00000 00000']
}

data2 = {
    'col1': ['A', 'D', 'E'],
    'col2': [1, 5, 8],
    'col3': ['X', 'Y', 'Z'],
    'col4': [10, 50, 30],
    'col5': [100, 100, 300],
    'col6': [1000, 2000, 3000],
    'col7': ['00100 00000 00000', '00100 00000 00000', '00100 00000 00000']
}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Step 1: 合併 df1 和 df2
combined_df = pd.concat([df1, df2])

# Step 2: 定義合併 col7 的邏輯運算函數
def merge_col7(col7_values):
    # 先移除空值
    col7_values = col7_values.dropna().unique()

    if len(col7_values) == 1:
        # 如果只有一個唯一值，返回它
        return col7_values[0]

    # 將每個 col7 的字串分割成位元串列表
    merged_bits = [0] * len(col7_values[0].replace(" ", ""))  # 初始化合併結果
    for col7 in col7_values:
        bits = col7.replace(" ", "")
        merged_bits = [str(int(b1) | int(b2)) for b1, b2 in zip(merged_bits, bits)]

    # 將合併後的結果重新加入空格並返回
    return ' '.join([''.join(merged_bits[i:i+5]) for i in range(0, len(merged_bits), 5)])

# Step 3: 以 col1 到 col6 作為分群條件，合併 col7
result_df = combined_df.groupby(['col1', 'col2', 'col3', 'col4', 'col5', 'col6'], as_index=False).agg({
    'col7': merge_col7
})
#(as_index=False 是一個選項，用來控制分組後的結果是否將分組的列設置為 DataFrame 的索引。)

# 顯示結果
print(result_df)


  col1  col2 col3  col4  col5  col6               col7
0    A     1    X    10   100  1000  10100 00000 00000
1    B     2    Y    20   200  2000  10000 00000 00000
2    C     3    Z    30   300  3000  10000 00000 00000
3    D     5    Y    50   100  2000  00100 00000 00000
4    E     8    Z    30   300  3000  00100 00000 00000


* 結果


在合併後，若兩筆資料的 col1 到 col6 相同，col7 就會進行位元的邏輯或（OR）合併，如 '10000 00000 00000' 和 '00100 00000 00000' 被合併成 '10100 00000 00000'。
其他不相同的資料則會保持不變並列入結果中

-----

In [ ]:
import pandas as pd

# 範例資料
data1 = {
    'col1': ['A', 'B', 'C'],
    'col2': [1, 2, 3],
    'col3': ['X', 'Y', 'Z'],
    'col4': [10, 20, 30],
    'col5': [100, 200, 300],
    'col6': [1000, 2000, 3000],
    'col7': ['10000 00000 00000', '10000 00000 00000', '10000 00000 00000']
}

data2 = {
    'col1': ['A', 'D', 'E'],
    'col2': [1, 5, 8],
    'col3': ['X', 'Y', 'Z'],
    'col4': [10, 50, 30],
    'col5': [100, 100, 300],
    'col6': [1000, 2000, 3000],
    'col7': ['00100 00000 00000', '00100 00000 00000', '00100 00000 00000']
}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Step 1: 合併 df1 和 df2
combined_df = pd.concat([df1, df2])

# Step 2: 定義合併 col7 的位元邏輯“或”運算
def bitwise_or_merge(col7_list):
    # 如果只有一個值，直接返回
    if len(col7_list) == 1:
        return col7_list[0]

    # 將每個 col7 的字串分割成位元串
    bit_lists = [bits.replace(" ", "") for bits in col7_list]

    # 進行逐位元的邏輯或運算
    merged_bits = ''.join(str(int(any([int(bits[i]) for bits in bit_lists]))) for i in range(len(bit_lists[0])))

    # 將結果重新加入空格，形成 5 位一組的形式
    return ' '.join([merged_bits[i:i+5] for i in range(0, len(merged_bits), 5)])

# Step 3: 對 col1 到 col6 相同的組合進行合併，合併 col7
result_df = combined_df.groupby(['col1', 'col2', 'col3', 'col4', 'col5', 'col6'], as_index=False)['col7'].agg(bitwise_or_merge)

# 顯示結果
print(result_df)


簡化的部分：
1. 邏輯清晰化：使用更直接的 apply 和 agg 方式來處理 col7 的位元合併，直接在分組聚合中處理邏輯運算。

2. 簡單的位元合併邏輯：

 - 將 col7 裡的位元字串去除空格，轉換為位元列表。
 - 使用 any() 函數在每個位元位置進行邏輯“或”運算，合併兩個位元串。
處理結果的格式化：

3. 最後的合併結果會重新插入空格，確保輸出的格式與原始數據相同。